In [2]:
import cv
import cv2
import time
import numpy as np
import numpy.ma as ma
import statistics

In [2]:
#Настройка и получение начальных параметров с камеры
def SetupCam():
    cam = cv2.VideoCapture(0)
    exposureTime = cam.get(cv2.CAP_PROP_EXPOSURE)
    iSO = cam.get(cv2.CAP_PROP_ISO_SPEED)
    aperture = cam.get(cv2.CAP_PROP_APERTURE)

    return  cam, exposureTime, iSO, aperture

In [ ]:
def GetExposureTriangle(cam):
    #Получение кадра
    ret, frame = cam.read()

    #Получение прараметров
    exposureTime = cam.get(cv2.CAP_PROP_EXPOSURE)
    iSO = cam.get(cv2.CAP_PROP_ISO_SPEED)
    aperture = cam.get(cv2.CAP_PROP_APERTURE)

    return  ret, frame, exposureTime, iSO, aperture

In [1]:
def GetLightFactor(frame, background, objects):

    #Получение светового канала
    imgHls = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS )
    lChannel = imgHls[:,:,1]

    #Использование задника и послдовательных кадров
    #Для получения всех контуров изменений
    fg_mask = background.apply(frame)
    fg_mask = cv2.medianBlur(fg_mask, 5)
    _, fg_mask = cv2.threshold(fg_mask,127,255,cv2.THRESH_BINARY)

    fg_mask_bb = fg_mask
    contours, hierarchy = cv2.findContours(fg_mask_bb,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2:]
    areas = [cv2.contourArea(c) for c in contours]

    maskedLChannel = lChannel

    if len(areas) >= 1:

        #Отобразить крупнейшие изменения в колличестве objects
        max_indexs = np.argsort(areas)[-objects:]

        for index in max_indexs:
            cv2.drawContours(frame,contours,index,(0,255,0),3)
            x,y,w,h = cv2.boundingRect(contours[index])
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)

            #Маска движущегося объекта накладываемая на световой канал
            #В итоге движущиеся объекты не влияют на вычисления освещенности
            mask = np.zeros(lChannel.shape,dtype=bool)
            mask[y:y+h, x:x+w] = True
            maskedLChannel = ma.masked_array(lChannel, mask)

    #Возвращаем среднее значение канала яркости
    return np.mean(maskedLChannel), frame

In [5]:
def show_webcam(mirror=True):

    #Начать видеозахват с параметрами
    cam = SetupCam()

    #Выполнение вычитания задего фона
    back_sub = cv2.createBackgroundSubtractorMOG2(history=700,
        varThreshold=25, detectShadows=True)


    # Определение свойств текста на экране
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontColor = (0, 255, 0)
    lineType = 2

    #Основной цикл работы с кадрами
    while True:
        #Получение кадра
        ret, frame, exposure, iso, aperture = cam.GetExposureTriangle()

        #Отзеркаливание для удобства
        if mirror:
            frame = cv2.flip(frame, 1)

        #Получение факторая яркости с учетом движений в кадре
        lightFactor, frame = GetLightFactor(frame, back_sub, 1)

        #Расчет освещенности
        Lux = (lightFactor*(aperture**2))/(exposure*iso)

        #Вывести на экран значение освещенности
        cv2.putText(
            frame,
            str(Lux),
            (10,300),
            font,
            fontScale,
            fontColor,
            lineType)

        cv2.imshow('my webcam', frame)
        if cv2.waitKey(1) == 27:
            break
    cam.release ()
    cv2.destroyAllWindows()

    return  Lux

In [6]:
Lux = show_webcam(mirror=True)